![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [9]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [11]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchmany(5))

[('airports',), ('airlines',), ('routes',)]


In [12]:
cursor.description

(('name', None, None, None, None, None, None),)

In [14]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(*cursor.fetchall(), sep = "\n") ## * is important. It turns the results in cursor.fetchall() seperately

('airports', 'table')
('ix_airports_index', 'index')
('airlines', 'table')
('ix_airlines_index', 'index')
('routes', 'table')
('ix_routes_index', 'index')


In [15]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [16]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [17]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [18]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [19]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [20]:
# Pragma is only useful for SQLite ONLY!
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [ ]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

In [ ]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [ ]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

In [9]:
statement = """ 
-- Select the unique list of countries with airports 
SELECT Distinct country
FROM airports
WHERE country LIKE "G%"
"""
cursor.execute(statement)
print(*cursor.fetchall(), sep = "\n")

('Greenland',)
('Ghana',)
('Germany',)
('Guernsey',)
('Gabon',)
('Gambia',)
('Guinea-Bissau',)
('Guinea',)
('Greece',)
('Gibraltar',)
('Guatemala',)
('Guam',)
('Guyana',)
('Guadeloupe',)
('Grenada',)
('Georgia',)


In [10]:
statement = """ 
-- Select the unique list of countries with airports 
SELECT Distinct country
FROM airports
WHERE country LIKE "%G%" OR country LIKE "%g%"
"""
#This shows that any countries with g insie
cursor.execute(statement) 
print(*cursor.fetchall(), sep = "\n")

('Papua New Guinea',)
('Greenland',)
('Algeria',)
('Ghana',)
('Nigeria',)
('Niger',)
('Togo',)
('Belgium',)
('Germany',)
('United Kingdom',)
('Guernsey',)
('Luxembourg',)
('Congo (Brazzaville)',)
('Congo (Kinshasa)',)
('Equatorial Guinea',)
('Madagascar',)
('Angola',)
('Gabon',)
('Gambia',)
('Guinea-Bissau',)
('Senegal',)
('Guinea',)
('Egypt',)
('Uganda',)
('Bulgaria',)
('Greece',)
('Hungary',)
('Portugal',)
('Bosnia and Herzegovina',)
('Gibraltar',)
('Montenegro',)
('Guatemala',)
('Nicaragua',)
('Tonga',)
('Afghanistan',)
('Argentina',)
('Guam',)
('Paraguay',)
('French Guiana',)
('Uruguay',)
('Guyana',)
('Antigua and Barbuda',)
('Guadeloupe',)
('Grenada',)
('Virgin Islands',)
('Anguilla',)
('Trinidad and Tobago',)
('British Virgin Islands',)
('Saint Vincent and the Grenadines',)
('Kyrgyzstan',)
('Bangladesh',)
('Hong Kong',)
('Singapore',)
('Mongolia',)
('Georgia',)
('Cocos (Keeling) Islands',)
('South Georgia and the Islands',)


### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [14]:
cursor.execute("""
SELECT DISTINCT a.latitude, a.longitude
FROM airports a
JOIN routes r 
    ON a.id = r.dest_id
""")
print(*cursor.fetchall(), sep='\n')

('-6.081689', '145.391881')
('-5.207083', '145.7887')
('-5.826789', '144.295861')
('-6.569828', '146.726242')
('-9.443383', '147.22005')
('-3.583828', '143.669186')
('61.160517', '-45.425978')
('64.190922', '-51.678064')
('67.016969', '-50.689325')
('76.531203', '-68.703161')
('65.659994', '-18.072703')
('65.283333', '-14.401389')
('66.058056', '-23.135278')
('63.985', '-22.605556')
('64.13', '-21.940556')
('46.485001', '-84.509445')
('51.391944', '-56.083056')
('49.1325', '-68.204444')
('48.330555', '-70.996391')
('64.298889', '-96.077778')
('49.950832', '-125.270833')
('49.91', '-99.951944')
('69.108055', '-105.138333')
('49.052333', '-123.870167')
('49.296389', '-117.6325')
('47.990833', '-66.330278')
('67.816667', '-115.143889')
('70.486111', '-68.516667')
('64.193333', '-83.359444')
('49.210833', '-57.391388')
('55.742333', '-120.183')
('53.309723', '-113.579722')
('61.094166', '-94.070833')
('68.304167', '-133.482778')
('63.75639', '-68.555832')
('45.868889', '-66.537222')
('54.6

('20.483831', '99.935353')
('16.907305', '96.133222')
('-5.061631', '119.554042')
('-1.190017', '136.107997')
('-3.368183', '135.496406')
('-4.528275', '136.887375')
('-2.576953', '140.516372')
('-4.102511', '138.957372')
('-8.520294', '140.418453')
('0.637119', '122.849858')
('37.469075', '126.450517')
('-0.918542', '119.909642')
('1.549447', '124.925878')
('-1.416753', '120.657669')
('0.831414', '127.381486')
('-1.038919', '122.771906')
('-3.710264', '128.089136')
('-2.920192', '132.267031')
('-3.644517', '133.695553')
('-2.532242', '133.438894')
('-0.891833', '134.049183')
('-0.926358', '131.121194')
('3.12385', '113.020472')
('1.484697', '110.346933')
('4.808303', '115.010439')
('4.1775', '114.321944')
('4.322014', '113.986806')
('2.261603', '111.985322')
('5.032247', '118.324036')
('5.937208', '116.051181')
('5.300683', '115.250181')
('4.313369', '118.121953')
('4.9442', '114.928353')
('0.460786', '101.444539')
('-6.125567', '106.655897')
('-0.874989', '100.351881')
('-1.816639', 

('52.2014', '-81.6969')
('56.0189', '-87.6761')
('56.5492', '-61.6803')
('63.3469', '-90.7311')
('51.4436', '-57.1853')
('52.9275', '-82.4319')
('60.0272', '-69.9992')
('53.5247', '-88.6428')
('48.6542', '-93.4397')
('51.7272', '-91.8244')
('-2.867', '-69.733')
('7.033', '-77.2')
('32.0708', '120.976')
('50.8308', '-58.9756')
('50.6744', '-59.3836')
('53.8056', '-78.9169')
('52.1133', '-94.2556')
('51.6586', '-85.9017')
('53.0125', '-89.8553')
('53.9656', '-91.0272')
('52.49', '-92.9711')
('52.8939', '-89.2892')
('53.8492', '-89.5794')
('52.7086', '-88.5419')
('63.1317', '-117.246')
('54.1797', '-58.4575')
('52.4525', '-125.303')
('16.1024', '-88.8083')
('17.7347', '-88.0325')
('8.71889', '-83.6417')
('70.7342', '-52.6962')
('72.7902', '-56.1306')
('66.9513', '-53.7293')
('77.4886', '-69.3887')
('60.9167', '-46.0586')
('60.14', '-45.2317')
('65.4125', '-52.9394')
('60.7158', '-46.0294')
('61.9922', '-49.6625')
('70.7433', '-22.6606')
('60.4644', '-45.5778')
('-5.6433', '143.895')
('-2.

In [13]:
cursor.execute("""
SELECT DISTINCT a.latitude, a.longitude
FROM airports a
JOIN routes r 
    ON a.id = r.source_id
""")
print(*cursor.fetchall(), sep='\n')

('-6.081689', '145.391881')
('-5.207083', '145.7887')
('-5.826789', '144.295861')
('-6.569828', '146.726242')
('-9.443383', '147.22005')
('-3.583828', '143.669186')
('61.160517', '-45.425978')
('64.190922', '-51.678064')
('67.016969', '-50.689325')
('76.531203', '-68.703161')
('65.659994', '-18.072703')
('65.283333', '-14.401389')
('66.058056', '-23.135278')
('63.985', '-22.605556')
('64.13', '-21.940556')
('46.485001', '-84.509445')
('51.391944', '-56.083056')
('49.1325', '-68.204444')
('48.330555', '-70.996391')
('64.298889', '-96.077778')
('49.950832', '-125.270833')
('49.91', '-99.951944')
('69.108055', '-105.138333')
('49.052333', '-123.870167')
('49.296389', '-117.6325')
('47.990833', '-66.330278')
('67.816667', '-115.143889')
('70.486111', '-68.516667')
('64.193333', '-83.359444')
('49.210833', '-57.391388')
('55.742333', '-120.183')
('53.309723', '-113.579722')
('61.094166', '-94.070833')
('68.304167', '-133.482778')
('63.75639', '-68.555832')
('45.868889', '-66.537222')
('54.6

('8.582294', '-71.161186')
('10.912926', '-63.967581')
('10.603117', '-66.990583')
('9.749067', '-63.1534')
('5.619992', '-67.606103')
('10.4805', '-68.073025')
('8.288528', '-62.760361')
('7.840831', '-72.439742')
('7.565111', '-72.035125')
('8.945147', '-64.151083')
('10.149733', '-67.9284')
('9.340797', '-70.584089')
('17.136749', '-61.792667')
('13.074603', '-59.492456')
('15.547028', '-61.3')
('14.591033', '-61.003175')
('18.099914', '-63.047197')
('16.265306', '-61.531806')
('12.004247', '-61.786192')
('18.337306', '-64.973361')
('17.701889', '-64.798556')
('18.494861', '-67.129444')
('18.255694', '-67.148472')
('18.008306', '-66.563028')
('18.439417', '-66.001833')
('17.311194', '-62.718667')
('14.020228', '-60.992936')
('13.733194', '-60.952597')
('12.501389', '-70.015221')
('12.131044', '-68.268511')
('12.188853', '-68.959803')
('17.496492', '-62.979439')
('18.040953', '-63.1089')
('18.204834', '-63.055084')
('11.149658', '-60.832194')
('10.595369', '-61.337242')
('18.444834',

('-8.93611', '-139.552')
('-9.35167', '-140.078')
('-9.768794', '-139.011256')
('-14.4281', '-146.257')
('-14.3114', '-178.066')
('-19.080028', '-169.925639')
('-17.269', '-178.976')
('-16.8028', '179.341')
('-12.4825', '177.071')
('-16.6906', '-179.877')
('-16.466749', '179.33986')
('-18.1992', '-178.817')
('-19.0581', '178.157')
('-17.7433', '-179.342')
('-19.8425', '-157.703')
('-20.1361', '-157.345')
('-21.8956', '-157.905')
('-19.9678', '-158.119')
('22.745561', '-74.182353')
('24.629417', '-75.673775')
('3.18495', '115.454')
('22.5132', '-78.511')
('9.957053', '-84.139797')
('8.53333', '-83.3')
('10.3557', '-85.852892')
('10.569', '-83.5148')
('7.51778', '-78.1572')
('19.475083', '-96.797506')
('15.2622', '-83.7812')
('18.5725', '-69.9856')
('19.267', '-69.742')
('21.333', '-71.2')
('21.4445', '-71.1423')
('26.5675', '12.8231')
('32.325', '15.061')
('39.5546', '27.0138')
('37.8554', '30.3684')
('37.7314', '38.4689')
('39.6546', '43.0271')
('37.539', '36.9534')
('40.562222', '43.1

In [15]:
cursor.execute("""
SELECT DISTINCT a.code, a.latitude, a.longitude
FROM airports a
JOIN routes r 
    ON a.id = r.source_id
GROUP BY code
""")
print(*cursor.fetchall(), sep = '\n')

('AAE', '36.822225', '7.809167')
('AAL', '57.092789', '9.849164')
('AAN', '24.261667', '55.609167')
('AAQ', '45.002097', '37.347272')
('AAR', '56.300017', '10.619008')
('AAT', '47.866667', '88.116667')
('AAX', '-19.563056', '-46.960278')
('AAY', '16.191667', '52.175')
('ABA', '53.74', '91.385')
('ABD', '30.371111', '48.228333')
('ABE', '40.652083', '-75.440806')
('ABI', '32.411319', '-99.681897')
('ABJ', '5.261386', '-3.926294')
('ABL', '67.106389', '-157.8575')
('ABM', '-10.950833', '142.459444')
('ABQ', '35.0402222', '-106.6091944')
('ABR', '45.4491', '-98.4218')
('ABS', '22.375953', '31.611722')
('ABT', '20.296139', '41.634277')
('ABV', '9.006792', '7.263172')
('ABX', '-36.067778', '146.958056')
('ABY', '31.5355', '-84.1945')
('ABZ', '57.201944', '-2.197778')
('ACA', '16.757061', '-99.753953')
('ACC', '5.605186', '-0.166786')
('ACE', '28.945464', '-13.605225')
('ACH', '47.485033', '9.560775')
('ACI', '49.706111', '-2.214722')
('ACK', '41.253053', '-70.060181')
('ACR', '-0.58', '-72.

('JLN', '37.151814', '-94.498269')
('JLR', '23.177817', '80.052047')
('JMK', '37.435128', '25.348103')
('JMU', '46.843394', '130.465389')
('JNB', '-26.139166', '28.246')
('JNG', '35.417', '116.533')
('JNN', '60.14', '-45.2317')
('JNS', '60.9167', '-46.0586')
('JNU', '58.354972', '-134.576278')
('JNX', '37.080556', '25.368056')
('JNZ', '41.1014', '121.062')
('JOE', '62.662906', '29.60755')
('JOG', '-7.788181', '110.431758')
('JOI', '-26.224453', '-48.797364')
('JOL', '6.05367', '121.011')
('JOS', '9.639828', '8.86905')
('JPA', '-7.148381', '-34.950681')
('JPR', '-10.8708', '-61.8465')
('JQA', '70.7342', '-52.6962')
('JQE', '7.51778', '-78.1572')
('JRO', '-3.429406', '37.074461')
('JSH', '35.216108', '26.101325')
('JSI', '39.1771', '23.503675')
('JSR', '23.1838', '89.160833')
('JST', '40.316111', '-78.833889')
('JSU', '65.4125', '-52.9394')
('JSY', '37.422792', '24.950936')
('JTC', '-22.157778', '-49.068333')
('JTR', '36.399169', '25.479333')
('JTY', '36.579886', '26.375822')
('JUB', '4.

('RAH', '29.626419', '43.490614')
('RAI', '14.9245', '-23.4935')
('RAJ', '22.309183', '70.779525')
('RAK', '31.606886', '-8.0363')
('RAO', '-21.134167', '-47.774189')
('RAP', '44.045278', '-103.057222')
('RAR', '-21.202739', '-159.805556')
('RAS', '37.325314', '49.605817')
('RBA', '34.051467', '-6.751519')
('RBQ', '-14.4275', '-67.498056')
('RBR', '-9.583', '-67.4836')
('RBV', '-8.16806', '157.643')
('RBY', '64.7272', '-155.47')
('RCB', '-28.741039', '32.092111')
('RCE', '48.608056', '-123.159722')
('RCH', '11.526222', '-72.925958')
('RCM', '-33.600556', '150.780833')
('RDC', '-8.033333', '-49.98')
('RDD', '40.509', '-122.293389')
('RDM', '44.254066', '-121.149964')
('RDN', '5.76528', '103.007')
('RDU', '35.877639', '-78.787472')
('RDV', '61.788056', '-157.350278')
('RDZ', '44.407869', '2.482672')
('REC', '-8.126794', '-34.923039')
('REG', '38.071206', '15.651556')
('REL', '-43.2105', '-65.270319')
('REN', '51.795786', '55.456744')
('REP', '13.410666', '103.81284')
('RES', '-27.449986'

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [27]:
# Which countries have the highest amount of active airports?
cursor.execute(""" 
SELECT country, COUNT(*) as c
FROM airlines
WHERE active = "Y"
GROUP BY country
ORDER BY COUNT(*) Desc
LIMIT 5;
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)


In [25]:
# Which countries have the highest amount of active airports?
cursor.execute(""" 
SELECT country, COUNT(name) as c
FROM airlines
WHERE active = "Y"
GROUP BY country
ORDER BY c Desc
LIMIT 5;
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)


In [26]:
# Which countries have the highest amount of inactive airports?
cursor.execute(""" 
SELECT country, COUNT(name) as c
FROM airlines
WHERE active = "N"
GROUP BY name
ORDER BY c Desc
LIMIT 5;
-- your code here;
""")
print(*cursor.fetchall(), sep='\n')

('United States', 5)
('United Kingdom', 5)
('Pakistan', 5)
('United States', 4)
('United Kingdom', 4)


In [29]:
# What about airports by timezones?
cursor.execute("""
-- your code here;
SELECT timezone, COUNT(*) as ct
FROM airports
GROUP BY timezone
ORDER BY ct DESC
LIMIT 5;
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)


In [30]:
import pandas as pd
pd.read_sql("""
-- your code here;
SELECT timezone, COUNT(*) as ct
FROM airports
GROUP BY timezone
ORDER BY ct DESC
LIMIT 5;
""", con) #the last part: con is the connection predefined before

,timezone,ct
0,America/New_York,628
1,America/Chicago,373
2,Europe/Berlin,319
3,America/Anchorage,258
4,Europe/Paris,232


In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [32]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [33]:
#shelter_data.loc[shelter_data.Name == "Max"]
max_data = shelter_data.query('Name == "Max" and Breed == "Pug Mix"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
43,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
289,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
378,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
925,A784016,Max,05/21/2019 03:13:00 PM,05/21/2019 03:13:00 PM,08/10/2018,Return to Owner,NaN,Dog,Intact Male,9 months,Staffordshire Mix,Brown Brindle/White
978,A681062,Max,06/12/2014 02:22:00 PM,06/12/2014 02:22:00 PM,06/11/2011,Return to Owner,NaN,Dog,Neutered Male,3 years,Jack Russell Terrier Mix,Brown/White
...,...,...,...,...,...,...,...,...,...,...,...,...
110969,A773678,Max,06/14/2018 02:30:00 PM,06/14/2018 02:30:00 PM,01/04/2018,Adoption,NaN,Dog,Neutered Male,5 months,Black/Tan Hound Mix,Brown/Black
111332,A785678,Max,12/10/2018 10:15:00 PM,12/10/2018 10:15:00 PM,02/07/2018,Return to Owner,NaN,Dog,Intact Male,10 months,German Shepherd Mix,Brown/Black
111344,A777049,Max,08/02/2018 12:23:00 PM,08/02/2018 12:23:00 PM,07/22/2017,Return to Owner,NaN,Dog,Intact Male,1 year,Siberian Husky Mix,White
111971,A769530,Max,04/21/2018 07:19:00 PM,04/21/2018 07:19:00 PM,04/06/2017,Adoption,NaN,Dog,Neutered Male,1 year,Miniature Poodle Mix,White/Cream


In [23]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A789027,Lennie,02/17/2019 11:44:00 AM,02/17/2019 11:44:00 AM,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
1,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
9,A789027,Lennie,03/10/2019 12:25:00 PM,03/10/2019 12:25:00 PM,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream
12,A795059,NaN,05/16/2019 10:14:00 AM,05/16/2019 10:14:00 AM,03/15/2019,Euthanasia,Rabies Risk,Dog,Intact Male,2 months,Pit Bull,Brown Brindle/White
13,A795043,NaN,05/16/2019 10:17:00 AM,05/16/2019 10:17:00 AM,03/31/2019,Euthanasia,Rabies Risk,Dog,Intact Male,1 month,German Shepherd,Black/Tan


## Goal 6: Transfering from sqlite to pandas

In [ ]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
-- your code here;
""", pd_con)
df

In [ ]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [34]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [39]:
pd.read_sql("""SELECT name, type FROM sqlite_master;""", con)
#print(*cursor.fetchall(), sep = "\n") 

,name,type
0,airports,table
1,ix_airports_index,index
2,airlines,table
3,ix_airlines_index,index
4,routes,table
5,ix_routes_index,index


In [42]:
pd.read_sql("""SELECT COUNT(*) FROM sqlite_master WHERE type = 'table';""", con)

,COUNT(*)
0,3


In [ ]:
# How many tables are in the database?

# your code here

In [45]:
# What's the primary key of each table?
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
results = cur.fetchall()
results

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [46]:
# What's the primary key of each table?
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
results = cur.fetchall()
for row in results:
    table = row[0]
    cur.execute(f"PRAGMA table_info({table})")
    columns =cur.fetchall()
    pk = ''
    for column in columns:
        if column[5] == 1:
            pk = column[1]
        print(f'Table {table} has primary key {pk}')
# your code here

Table Album has primary key AlbumId
Table Album has primary key AlbumId
Table Album has primary key AlbumId
Table Artist has primary key ArtistId
Table Artist has primary key ArtistId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Customer has primary key CustomerId
Table Employee has primary key EmployeeId
Table Employee has primary key EmployeeId
Table Employee has primary key EmployeeId
Table Employee has primary key EmployeeId
Table Employee has primary key EmployeeId
Table Employee has primary key EmployeeId
Table Employee has

In [47]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
results = cur.fetchall()
for row in results:
    table = row[0]
    cur.execute(f"PRAGMA table_info({table})")
    columns = cur.fetchall()
    pk = ''
    for column in columns:
        if column[5] == 1:
            pk = column[1]
    print(f'Table {table} has primary key {pk}')

Table Album has primary key AlbumId
Table Artist has primary key ArtistId
Table Customer has primary key CustomerId
Table Employee has primary key EmployeeId
Table Genre has primary key GenreId
Table Invoice has primary key InvoiceId
Table InvoiceLine has primary key InvoiceLineId
Table MediaType has primary key MediaTypeId
Table Playlist has primary key PlaylistId
Table PlaylistTrack has primary key PlaylistId
Table Track has primary key TrackId


In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [54]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
-- your code here;
SELECT Name, sum(Quantity) as c
FROM track t
JOIN InvoiceLine il on t.TrackId = il.TrackId
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

('The Trooper', 5)
('Eruption', 4)
('Hallowed Be Thy Name', 4)
('Sure Know Something', 4)
('The Number Of The Beast', 4)
('Untitled', 4)
('2 Minutes To Midnight', 3)
('Blood Brothers', 3)
('Brasil', 3)
('Can I Play With Madness', 3)


In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

## Reflection

## Learning goals review:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1O0dMGqjIV6-JIDYSIeGqTjIdep1BFiIVCc29UbEgwUM/edit)!